# Definir Queries Útiles

En esta notebook vamos a revisar una propuesta del proceso para definir queries que obtengan un volumen suficiente de tweets constantemente. Se usará de ejemplo el proceso realizado para el proyecto del Laboratorio de Migración.

Para ello, se verán los siguientes puntos:
- Identificar lenguajes y regiones a minar
- Obtener un primer dataset
- N-Grams para expandir queries
- Proceso iterativo

Nota: Para este proceso nos interesa obtener un volumen considerable de tweets, por lo que entre más días se puedan minar, mejor. En caso de contar con acceso [*Essential* o *Elevated*](https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-level) se recomienda [aplicar de manera gratuita a credenciales Académicas](https://developer.twitter.com/en/products/twitter-api/academic-research). Esta notebook se hacen queries dando por hecho que las credenciales son académicas. En el proyecto de Laboratorio de Migración se pudo obtener sin mayor problema.

In [1]:
import os
import json
import pendulum
import pandas as pd

from tqdm import tqdm

from twarc.client2 import Twarc2
from twarc.expansions import TWEET_FIELDS
from twarc.expansions import ensure_flattened

Nota: Estas son funciones desarrolladas por el equipo de entropia.ai, el código está en el directorio de helpers.

In [2]:
from helpers.clean_tweets import clean_tweets_dataset
from helpers.get_n_grams import get_n_grams

In [3]:
CREDENTIALS_TWITTER_API = {
    'bearer_token': "Enter your own Bearer token",

    # 'api_key': "Enter your own API Key",
    # 'api_secret_key': "Enter your own API Secret Key",
    # 'access_token': "Enter your own access_token",
    # 'access_token_secret': "Enter your own access_token_secret"
}

IS_ACADEMIC = False # Cambiar a True, si las credenciales son Academicas

Esta función es la que se encarga de minar los tweets, dada una lista de queries y un intervalo de tiempo. La notebook 1_Fundamentos_Twitter_API.ipynb contiene más detalles de qué es y como funciona la API de Twitter.

In [5]:
def get_tweets(credentials_api,
               queries_list,
               output_file,
               since_date, until_date,
               is_academic=False):
    """Function in charge of scrape tweets from the 
    official Twitter API, using the library named Twarc.

    Args:
        credentials_api (dict): Dictionary with the Twitter API credentials.
        queries_list (list[str]): List of queries to scrape.
        output_file (str): Path to the file in which to store the results.
        since_date (datetime): Start of the time span to scrape.
        until_date (datetime): End of the time span to scrape.
        is_academic (bool, optional): If the credentials has Research 
                                      Academic access level.
    """

    # Instiate the Twarc Client
    twarc_client = Twarc2(**credentials_api)

    # Make some tweaks for using the research credentials
    max_size = 100
    tweet_fields = TWEET_FIELDS.copy()
    search_func = twarc_client.search_recent
    if(is_academic):
        search_func = twarc_client.search_all
        max_size = 500

        # Remove the context_annotations attr to
        # scrape 500 tweets per request
        tweet_fields.remove('context_annotations')

    tweet_fields = ','.join(tweet_fields)

    with open(output_file, 'a') as pages_file:
        for query in tqdm(queries_list):

            search_results = search_func(query=query,
                                         start_time=since_date,
                                         end_time=until_date,
                                         tweet_fields=tweet_fields,
                                         max_results=max_size)

            # Write all the obtained tweets
            for page in search_results:

                # Write one by one the tweets
                for tweet in ensure_flattened(page):
                    json.dump(tweet, pages_file, ensure_ascii=False)
                    pages_file.write('\n')

## Primer Dataset

### Identificar países y lenguajes de interés
Un primer paso es identificar en que lenguaje y países nos interesan minar, en el caso del Laboratorio de Migración se minan tweets de los países hispanohablantes de LATAM (español), países del caribe (inglés) y Brasil (portugués). Dependiendo de que tanto se habla ese idioma en el mundo y los países que nos interesa cubrir, van a cambiar que tan específicos o generales pueden que ser los queries y qué operadores usaríamos.

### Ejemplos proyecto Laboratorio de Migración

### Inglés
Tweets en inglés se publican desde todas partes del mundo y en este caso, solo nos interesan ciertos países que por densidad de población no pueden generar un gran volumen de tweets. Por lo que si no usamos el operador `place_country`, vamos a obtener tweets de todo el mundo y eso significaría invertir más tiempo y recursos en el minado, procesamiento, etc. Para evitar esto, desde ya se puede definir que usaremos el operador `place_country` para así limitar la búsqueda de tweets a esos países y esto nos dará la oportunidad de usar keywords con un alcance más abierto. 

En el ejemplo de migración se podrían usar: `migrants, immigrants, emigrants, migration, immigration, emigration, migratory, immigratory y emigratory` y luego añadir variaciones como `pro-migrants, anti-migrants, ...`

Query ejemplo: `(migrants OR immigrants OR emigrants OR migration OR immigration OR emigration OR migratory OR immigratory OR emigratory) (place_country:BB OR place_country:GY OR place_country:BZ OR place_country:JM OR place_country:HT OR place_country:SR)`

### Portugués
En este caso, los tweets en portugués van a ser principalmente de Brasil o de Portugal, por lo que se podría aplicar el operador `place_country` para obtener solo tweets publicados desde Brasil o minar queries sin ninguna restricción para luego filtrarlos basándose en sí son geo-localizables a otro lado que no sea Brasil. 

Palabras claves podrían ser: `migratórias, migração, migrações, migrantes, migradas`

Query ejemplo: `(migratórias OR migração OR migrações OR migrantes OR migradas) place_country:BR`

### Español
Para este lenguaje, exceptuando a España, nos interesa obtener tweets del conjunto de países que van a generar la mayoría de tweets en español. Y aunque es buena práctica utilizar queries más general junto con el operador `place_country`, esto nos dejaría con un volumen muy inferior y que no terminaría de reflejar las conversaciones reales de lo que se podría obtener.

Por lo que aquí conviene hacer ambas cosas, tener queries con keywords generales, pero cerrado a solo los países de interés y tener queries con keywords más específicos, pero sin ninguna limitación de geolocalización, para luego tratar de extraer el país de origen por otros atributos (Esto se va a revisar en la notebook llamada 3_Variables_Demograficas.ipynb).

Query ejemplo: `(migrar OR inmigrar OR emigrar OR migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes OR migratorios OR inmigratorios OR emigratorios OR migratorias OR inmigratorias OR emigratorias) (place_country:MX OR place_country:AR OR place_country:AR)`.

Esto se replicaría para todos los países que nos interesa y nos serviría como primer dataset para iterar sobre los queries. No solo para los queries con el filtro por país, también para aquellos que no lo tienen, ya que esos debe de ser más específicos para no traernos demasiado ruido, pero que cubran la mayor parte de la conversación y así tener un volumen suficiente.

Para esta notebook usaremos de ejemplo la definición de queries en español, y como se mencionó previamente, el primer paso es definir una lista preliminar de keywords y concatenarlas con el operador `OR`. Optar por palabras que de manera explícita hable del tema de interés.

In [6]:
list_keywords = ["migrar", "inmigrar", "emigrar", 
                 "migrante", "inmigrante", "emigrante", 
                 "migrantes", "inmigrantes", "emigrantes", 
                 "migratorios", "inmigratorios", "emigratorios", 
                 "migratorias", "inmigratorias", "emigratorias"]
list_keywords = ' OR '.join(list_keywords)

En seguida hay que añadir los operadores para filtrar por países. Esto nos ayudará a encontrar keywords o expresiones que estén atadas al contexto de cada país, ayudando así a filtrar un poco la conversación migratoria.

También se añade el operador `-is:retweet` y `lang:es`, evitando traer retweets, ya que por el momento solo nos interesa obtener los tweets únicos y que estén en español.

In [7]:
lst_queries = f"({list_keywords}) (place_country:MX OR place_country:AR OR place_country:CO) lang:es -is:retweet"
lst_queries

'(migrar OR inmigrar OR emigrar OR migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes OR migratorios OR inmigratorios OR emigratorios OR migratorias OR inmigratorias OR emigratorias) (place_country:MX OR place_country:AR OR place_country:CO) lang:es -is:retweet'

Definimos un intervalo de tiempo considerable para obtener un volumen útil de tweets. En el proyecto del Laboratorio de Migración se empezó con minar tweets de todo el año 2019 y 2020, al ser tanto tiempo, podremos obtener tweets de momentos en los que ocurrieran eventos relacionados con migración, así como en momentos donde sea un tema menos viral. 
Para este ejemplo usaremos todo lo que va del 2022.

Nota: Todos los archivos que se obtengan durante el desarrollo de esta notebook se van a encontrar en el directorio: "./files"

In [8]:
lst_queries = [lst_queries]

date_start = pendulum.datetime(year=2022, month=1, day=1)
date_end = pendulum.datetime(year=2022, month=9, day=10)

file_tweets = os.path.abspath("./files/1_dataset.jsonl")

Se ejecuta la función para minar los tweets

In [9]:
get_tweets(credentials_api = CREDENTIALS_TWITTER_API,
           queries_list = lst_queries,
           output_file = file_tweets,
           since_date = date_start, 
           until_date = date_end,
           is_academic= IS_ACADEMIC)

100%|██████████| 1/1 [01:27<00:00, 87.22s/it]


Vemos cuantos tweets obtuvimos

In [10]:
!wc -l ./files/1_dataset.jsonl

8769 ./files/1_dataset.jsonl


Estos 8,593 tweets nos van a servir como primer dataset para identificar keywords nuevas que estén relacionadas con el fenómeno que queremos estudiar, ya sea que se refieran a el de manera:
- Explicita: "migrante" o "inmigrantes" 
- Implícita: "los mexicanos en estado unidos" o "las personas ilegales"

### N-Grams

Estas son un concepto muy utilizado en el procesamiento de lenguaje natural (NLP, por siglas en inglés), y se pueden entender como una secuencia de N palabras, por ejemplo:
- Dream on (2-Grama)
- Kickstart My Heart (3-Grama)
- Hail to the King (4-Grama)

Estas secuencias de texto se pueden contabilizar en un corpus de texto y obtener la frecuencia de aparición de una palabra tras la otra, y así saber qué palabras puede considerarse como una sola entidad. Por ejemplo, en un texto sobre la historia de la banda de rock Aerosmith, el 2-grama "Dream On" se puede considerar una sola palabra, dado que se refiere a la canción que sirvió como soundtrack de la película  Armagedon. También puede servir para realizar una predicción de qué palabras siguen, como el autocompletado que realiza Spotify para mostrar una lista de opciones de la canción original y covers cuando solo escribimos "Kick Start...".

Podemos considerar los N-Grams como una herramienta útil para encontrar patrones entre los tweets que hablan sobre migración y así extender la lista de keywords y/o formar queries que se apoyen en que un tweet contenga al menos dos keywords.

Primero hay que preprocesar los tweets para así pasar todo a minúsculas y eliminar partes del texto que no sean de utilidad:
- URLs
- Números
- Emojis
- Caracteres especiales
- Risas (Jajaj o Hahaha)
- Espacios en blanco múltiples
- Mención a usuarios 
- Borrar stopwords (que, como, cuál, etc)

Para lograr hacer esto primero hay que cargar los identificadores únicos (IDs) y textos de los tweets. 

In [12]:
df_tweets = pd.read_json("./files/1_dataset.jsonl", lines=True)
df_tweets = df_tweets.drop_duplicates(subset=['text']).reset_index(drop=True)
df_tweets = df_tweets[['id', 'text']]

In [16]:
df_tweets = clean_tweets_dataset(df_tweets, doc_column='text', clean_col='text_preprocessed')
df_tweets[['text', 'text_preprocessed']][200:220]

,text,text_preprocessed
200,"Derechos humanos, respeto, unión, pluralismo, ...",derechos humanos respeto union pluralismo dive...
201,Nadie que no lo haya experimentado en carne pr...,experimentado carne entender realmente necesit...
202,"Un hogar lejos del hogar, ojalá todos los migr...",hogar hogar migrantes puedan encontrar destino...
203,Hace cuatro años decidí migrar. En todo este t...,anos decidi migrar argentina convertido aprend...
204,Muy feliz día del inmigrante a todos los que f...,feliz inmigrante forjaron nacion eligen patria...
205,#FiestaNacionalDelInmigranteEnObera #inmigrant...,fiestanacionaldelinmigranteenobera inmigrantes...
206,"Además, entregamos un reconocimiento a la fami...",entregamos reconocimiento familia miguel balut...
207,"#Comodoro es un crisol de razas ☀️, la tierra ...",comodoro crisol razas tierra inmigrantes eligi...
208,🗓️4/09 - 𝘿𝙄𝘼 𝘿𝙀𝙇 𝙄𝙉𝙈𝙄𝙂𝙍𝘼𝙉𝙏𝙀\nDesde @Juventud_S...,saludar todxs lxs trabajadorxs migrantes forma...
209,Desde la mesa interministerial saludamos a tod...,mesa interministerial saludamos todes migrante...


Obtener bigramas

In [17]:
srs_bigrams = get_n_grams(df_tweets['text_preprocessed'], 
                          n_gram = 2, 
                          n_top = 200)
srs_bigrams[:20]

/home/noecampos/.pyenv/versions/BID/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,n_gram,total_freq
0,poblacion migrante,252
1,migrantes venezolanos,229
2,personas migrantes,145
3,prov inmigrante,133
4,brenas capital,132
5,capital prov,132
6,derechos humanos,99
7,migrantes refugiados,94
8,mujeres migrantes,79
9,lasbrenas brenas,78


Obtener trigramas

In [18]:
srs_trigramas = get_n_grams(df_tweets['text_preprocessed'], 
                            n_gram = 3, 
                            n_top = 200)
srs_trigramas[:20]

/home/noecampos/.pyenv/versions/BID/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,n_gram,total_freq
0,brenas capital prov,132
1,capital prov inmigrante,132
2,lasbrenas brenas capital,78
3,barrio grave gracias,76
4,fomentan sosobra intimidan,76
5,forasteros inmigrantes fomentan,76
6,gracias loscontenedoresdebasura unico,76
7,grave gracias loscontenedoresdebasura,76
8,inmigrantes fomentan sosobra,76
9,inseguridad barrio grave,76


Explorando esta lista de n-gramas encontraremos que:
- Las palabras "migrante", "inmigrante" o "emigrante" viene acompañadas de:
  - niños
  - población
  - personas
  - forasteros
  - inseguridad
  - refugiados
  - caravana
  - ilegales
  - plaza
  - venezolano
  - hermanos
  - familias
  - mujer
  - atención
  - grupo
  - países
  - retornados
  - miles
  - millones
  - tráfico


- Las palabras "migratorio" y "migratoria" vienen acompañadas de:
  - leyes
  - política
  - temas
  - pacto
  - reformas
  - crisis
  - fenómeno

## Primer query basado en N-Grams
Con el grupo de palabras listadas previamente, se construye un primer query.

In [17]:
first_query = "(migrante OR inmigrante OR emigrante) (niño OR mujer OR ilegal OR muerto OR persona OR hermano OR forastero OR refugiado OR retornado OR familia OR caravana OR grupo OR población OR pais OR miles OR millones OR plaza OR trafico OR inseguridad OR atención OR venezolano)"

A este primer query se le añade variaciones de las palabras, por ejemplo:
- migrante ⨪> migrantes, inmigrante, ...
- niño -> niños, niña, niñas
- caravana -> caravanas
- refugiado -> refugiados, refugiada, refugiadas

In [18]:
first_query = "(migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes) (niño OR niños OR niña OR niñas OR mujer OR mujeres OR hombre OR hombres OR ilegal OR ilegales OR muerto OR muertos OR persona OR personas OR hermano OR hermanos OR hermana OR hermanas OR forastero OR forasteros OR forastera OR forasteras OR refugiado OR refugiados OR refugiada OR refugiadas OR familia OR familias OR caravana OR caravanas OR grupo OR grupos OR población OR poblaciones OR pais OR paises OR miles OR millones OR plaza OR plazas OR trafico OR inseguridad OR atención OR venezolano)"

Esto se repite para las palabras migratorio y migratoria

In [19]:
second_query = "(migratorio OR migratoria OR migratorios OR migratorias OR inmigratorio OR inmigratoria OR inmigratorios OR inmigratorias OR emigratorio OR emigratoria OR emigratorios OR emigratorias) (ley OR leyes OR politica OR politicas OR tema OR temas OR pacto OR pactos OR reforma OR reformas OR fenomeno OR crisis)"

Aprovechando que estamos trabajando con credenciales con acceso academico, podriamos incluso juntar estos dos queries con un OR para así ahorrar tiempo y requests a la API.

In [20]:
merge_query = f"({first_query}) OR ({second_query})"

Añadimos el operador de `has:geo` pero negado: `-has:geo` y los operadores para evitar retweets y filtrar al español. Con esto vamos a obtener tweets que cumplan con los keywords y que no tengan información de geolocalización. Esto porque aquellos que tengan información de alguna ciudad o país, ya nos los trajimos con los queries que tienen el operador `place_country`. 

Para el proyecto del Laboratorio de Migración se hizo de esta manera, y aquellos tweets que no tuvieran información de geolocalización dada por Twitter se intentó inferir por medio de los atributos, *description* y *location* del usuario. Esto se verá en la siguiente notebook llamada "3_Get_Location.ipynb".

In [21]:
merge_query = f"({first_query}) OR ({second_query}) lang:es -is:retweet -has:geo"
print(merge_query)

((migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes) (niño OR niños OR niña OR niñas OR mujer OR mujeres OR hombre OR hombres OR ilegal OR ilegales OR muerto OR muertos OR persona OR personas OR hermano OR hermanos OR hermana OR hermanas OR forastero OR forasteros OR forastera OR forasteras OR refugiado OR refugiados OR refugiada OR refugiadas OR familia OR familias OR caravana OR caravanas OR grupo OR grupos OR población OR poblaciones OR pais OR paises OR miles OR millones OR plaza OR plazas OR trafico OR inseguridad OR atención OR venezolano)) OR ((migratorio OR migratoria OR migratorios OR migratorias OR inmigratorio OR inmigratoria OR inmigratorios OR inmigratorias OR emigratorio OR emigratoria OR emigratorios OR emigratorias) (ley OR leyes OR politica OR politicas OR tema OR temas OR pacto OR pactos OR reforma OR reformas OR fenomeno OR crisis)) -has:geo lang:es -is:retweet


Dado que minar todo lo que va del 2022 tomaría mucho tiempo, aprovecharemos que la API de Twitter nos permite hacer un conteo de tweets dado un query, para así saber cuantos tweets obtendríamos.

In [22]:
twarc_client = Twarc2(**CREDENTIALS_TWITTER_API)
counts_per_month = twarc_client.counts_all(query=merge_query,
                                           start_time=date_start,
                                           end_time=date_end,
                                           granularity='day')

df_all = []
for month_count in tqdm(counts_per_month): 
    df_all.append(pd.DataFrame(month_count['data']))
    
df_all = pd.concat(df_all)

9it [00:17,  1.98s/it]


In [23]:
print(f"Si se minaran lo que va del año 2022 se obtendrian {df_all['tweet_count'].sum()} tweets")

Si se minaran lo que va del año 2022 se obtendrian 2654060 tweets


Pero aun así se puede hacer el ejercicio de minar un par de días

In [24]:
lst_queries = [merge_query]

date_start = pendulum.datetime(year=2022, month=9, day=9)
date_end = pendulum.datetime(year=2022, month=9, day=11)

file_tweets = os.path.abspath("./files/2_dataset.jsonl")

In [25]:
get_tweets(credentials_api = CREDENTIALS_TWITTER_API,
           queries_list = lst_queries,
           output_file = file_tweets,
           since_date = date_start, 
           until_date = date_end,
           is_academic= IS_ACADEMIC)

100%|██████████| 1/1 [02:12<00:00, 132.03s/it]


In [26]:
!wc -l ./files/2_dataset.jsonl

13680 ./files/2_dataset.jsonl


Con minar solo un par de días, se obtuvo mucho más volumen que en todo el año, con el conjunto de queries abiertos pero filtrado a ciertos países. Mientras que si mináramos todo lo que va del año obtendríamos 2,654,167 tweets únicos.

## Más keywords

Uno de los objetivos del proyecto Laboratorio de Migración es llevar un track de la conversación xenófoba dentro de la conversación migrante. Es decir, tweets que estén atacando e insultado a los migrantes, refugiados, etc. Con esto se le dará más profundidad al estudio de la conversación de la migración, dado que ahora también se podría estimar que porcentaje de ella es con un tono xenófobo.

Para asegurar que se obtendran tweets en ese tono, se hizo el mismo ejercicio de queries abiertas y filtros por paises pero ya no solo con keywords abiertos, sino que usando operadores logicos se concatenaron con palabras que fueran racistas o xenofobas. Y se aplico el proce de N-Grams para extender esta lista de palabras.

Por ejemplo, buscar tweets que contengan la palabra `migrante` y variaciones pero concatenadas con un `AND` a palabras como `ilegales, violadores, ladrones, terroristas`.

Query ejemplo: `((migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes) (ilegales OR ilegal OR violadores OR violador OR ladrones OR ladron OR terroristas OR terrorista)) (place_country:MX OR place_country:AR OR place_country:CO)`

Primero vamos a minar nuevos tweets

In [27]:
new_query = "(migrar OR inmigrar OR emigrar OR migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes OR migratorios OR inmigratorios OR emigratorios OR migratorias OR inmigratorias OR emigratorias) (malditos OR maldito OR ilegales OR ilegal OR violadores OR violador OR ladrones OR ladron OR terroristas OR terrorista)"
new_query = f"({new_query}) (place_country:MX OR place_country:AR OR place_country:CO) lang:es -is:retweet"
new_query

'((migrar OR inmigrar OR emigrar OR migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes OR migratorios OR inmigratorios OR emigratorios OR migratorias OR inmigratorias OR emigratorias) (malditos OR maldito OR ilegales OR ilegal OR violadores OR violador OR ladrones OR ladron OR terroristas OR terrorista)) (place_country:MX OR place_country:AR OR place_country:CO) lang:es -is:retweet'

Nota: Por cuestiones de volumen, se minara todo 2020, 2021 y 2022.

In [28]:
new_query = [new_query]

date_start = pendulum.datetime(year=2020, month=1, day=1)
date_end = pendulum.datetime(year=2022, month=9, day=10)

file_tweets = os.path.abspath("./files/3_dataset.jsonl")

get_tweets(credentials_api = CREDENTIALS_TWITTER_API,
           queries_list = new_query,
           output_file = file_tweets,
           since_date = date_start, 
           until_date = date_end,
           is_academic= IS_ACADEMIC)

100%|██████████| 1/1 [00:11<00:00, 11.33s/it]


In [29]:
!wc -l ./files/3_dataset.jsonl

683 ./files/3_dataset.jsonl


#### Extraer N-Grams al nuevo dataset

Nuevamente, se utiliza la técnica de N-Grams para obtener queries que sirvan para minar tweets, ahora siendo sobre migración y xenófobos.

In [30]:
df_tweets = pd.read_json("./files/3_dataset.jsonl", lines=True)
df_tweets = df_tweets[['id', 'text']]

df_tweets = clean_tweets_dataset(df_tweets, doc_column='text', clean_col='text_preprocessed')

srs_bigrams = get_n_grams(df_tweets['text_preprocessed'], 
                          n_gram = 2, 
                          n_top = 200)

/home/noecampos/.pyenv/versions/BID/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Y con esto se consiguieron nuevas palabras, por ejemplo:
- delincuentes
- mafiosos
- ignorantes
- crimen organizado
- agresores
- crimen
- traficantes
- drogadictos

Con estas nuevas palabras se puede armar un nuevo query para el minado sin restricción por países

In [31]:
new_query = "(migrante OR inmigrante OR emigrante OR migrantes OR inmigrantes OR emigrantes) (malditos OR maldito OR ilegales OR ilegal OR violar OR violadores OR violador OR ladrones OR ladron OR terrorismo OR terroristas OR terrorista OR delincuencia OR delincuentes OR delincuente OR mafia OR mafiosos OR mafioso OR ignorantes OR ignorante OR agredir OR agresores OR agresor OR crimen OR criminal OR criminales OR trafico OR traficantes OR traficante OR drogadictos OR drogradictas OR drogradicto OR drogradicta)"
new_query = f"({new_query}) -has:geo lang:es -is:retweet"

date_start = pendulum.datetime(year=2022, month=1, day=1)
date_end = pendulum.datetime(year=2022, month=9, day=10)

twarc_client = Twarc2(**CREDENTIALS_TWITTER_API)
counts_per_month = twarc_client.counts_all(query=new_query,
                                           start_time=date_start,
                                           end_time=date_end,
                                           granularity='day')

df_all = []
for month_count in tqdm(counts_per_month): 
    df_all.append(pd.DataFrame(month_count['data']))
    
df_all = pd.concat(df_all)

9it [00:18,  2.01s/it]


In [32]:
print(f"Si se minaran lo que va del año 2022 se obtendrian {df_all['tweet_count'].sum()} tweets")

Si se minaran lo que va del año 2022 se obtendrian 142994 tweets


Este nuevo query obtendría 142,994 tweets en lo que va del año. Hay que tener en cuenta que probablemente haya tweets que este query traiga tweets duplicados que existan dentro de los posibles 2 millones de tweets previamente contados, pero es muy fácil lidiar con tweets duplicados dado que todos tienen un ID único.

## Iterar el ejercicio

Este ejercicio se tiene que repetir con cada dataset que vayamos generando hasta que consideramos que generamos un volumen suficiente. No hay una métrica escrita en piedra, por lo que es cuestión de iterar múltiples veces, añadiendo y quitando keywords. Algo útil y rápido para ver si nuestras keywords/operadores obtienen tweets es [utilizar la búsqueda avanzada dentro del portal de Twitter](https://help.twitter.com/es/using-twitter/twitter-advanced-search), con ella podremos tener una vista rápida de si obtiene tweets o no.

### Lidiar con falsos positivos

No importa cuan especificos los queries, simpre habra que lidiar con tweets que no son del tema que tratamos de estudiar. Por lo que hay que implementar algunos filtros más para descartar ese subconjunto de tweets. En el caso del proyecto del Laboratorio, se opto por entrenar un modelo de clasificación binaria para decidir si los tweets recibidos son o no sobre migración.